In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 43.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.1.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 18.1.0 which is inc

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize MediaPipe Drawing Utils for visualization (optional)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles # Import drawing styles


def calculate_eye_aspect_ratio(landmarks, eye_indices):
    """Calculates the Eye Aspect Ratio (EAR)."""
    try:
        eye_points = np.array([(landmarks.landmark[i].x, landmarks.landmark[i].y) for i in eye_indices])

        A = np.linalg.norm(eye_points[1] - eye_points[5])
        B = np.linalg.norm(eye_points[2] - eye_points[4])
        C = np.linalg.norm(eye_points[0] - eye_points[3])

        ear = (A + B) / (2.0 * C)
        return ear
    except IndexError:
        return None

def assess_eye_contact(frame):
    """Assesses eye contact based on EAR."""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye_indices = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398]
            right_eye_indices = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

            left_ear = calculate_eye_aspect_ratio(face_landmarks, left_eye_indices)
            right_ear = calculate_eye_aspect_ratio(face_landmarks, right_eye_indices)

            if left_ear is not None and right_ear is not None:
                ear = (left_ear + right_ear) / 2.0

                if ear < 1.37:
                    eye_contact_status = "Low Eye Contact (Potential Blink/Closed Eyes)"
                else:
                    eye_contact_status = "Good Eye Contact"

                

                 # Manually define drawing styles (if get_default_face_mesh_landmarks_style() is not available)
                landmark_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
                connection_drawing_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1)
        
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=landmark_drawing_spec, 
                    connection_drawing_spec=connection_drawing_spec)

                cv2.putText(frame, eye_contact_status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                return frame, eye_contact_status, ear
            else:
                return frame, "Face Not Detected Properly", None

    return frame, "No Face Detected", None


# Example usage with webcam
cap = cv2.VideoCapture('/kaggle/input/eyes-score/WIN_20241212_16_29_38_Pro.mp4')
count= 0
eye_score_total= 0.0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame, eye_contact_status, ear = assess_eye_contact(frame)

    count=count+1
    eye_score_total=eye_score_total+ear
    plt.imshow(frame)
    plt.show()



cap.release()
# cv2.destroyAllWindows()
face_mesh.close()

In [ ]:
ear_final=eye_score_total/count
print(ear_final)
if ear_final < 1.37:
    eye_contact_status = "Low Eye Contact (Potential Blink/Closed Eyes/No Eye Contact)"
else:
    eye_contact_status = "Good Eye Contact"
print(eye_contact_status)